## A common pipeline to clean dataframes


0. Import modules
1. Normalize column names
2. Eval misspelling in object non-date columns
3. Eval & Drop duplicates without index_id `mlg.DF_wo_colX(df, colnames)`
4. Check NAs `mlg.na_absperc(df)`
    - By Columns
    - By Rows
5. Eval 'unique/freq ratio'`mlg.categ_summ(df)`
6. Eval Data types
7. Save the cleaned dataframe

## Summary of cleaning `language.csv`

I made no changes!!

**Original language.csv** had 6 rows and 3 columns `['language_id', 'name', 'last_update'`

* 1. No column name to normalize
* 2. No misspelling values
* 3. No hidden duplicated rows
* 4. No NA's
* 5. Nothing striking from unique/freq ratio analysis
* 6. No data type to transform
* 7. Cleaned DF saved as `clean/language1.csv`

**Cleaned language1.csv** had 6 rows and 3 columns `['language_id', 'name', 'last_update'`

### 0. Import modules

First things first!!

We need to import modules and set default notebook properties as;

- leading with warnings or
- defining how we want to display the outputs and plots.

I will also import my own modulemlgfrom scr/dataanalysis1.py.
When I make modifications in my functions, I need to detatch the module and load it again!

In [1]:
# Import modules etc

import pandas as pd
import numpy as np
import re

np.random.seed(42)
pd.set_option('display.max_columns', None) # show all the columns

# print the plot in the jupyter output
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore') # ignorar warnings

import pylab as plt   # import matplotlib.pyplot as plt
import seaborn as sns
import fuzzywuzzy as fzw

# Import my module
from src import dataanalysis_fun1 as mlg 

In [2]:
# Reload my module if neccessary

import importlib
from src import dataanalysis_fun1 as mlg # Import the module
#importlib.reload(mlg)  # Reload the module

# Suppress warning when reloading the module
with warnings.catch_warnings():
    warnings.simplefilter("ignore") 
    importlib.reload(mlg)  # Reload the module

In [3]:
DF_raw = pd.read_csv('../../data/raw/language.csv')

### 1. Normalize column names

In [4]:
display(DF_raw.columns)
setA = set(DF_raw.columns)
DF_raw=mlg.colnnam_clean(DF_raw)
setB = set(DF_raw.columns)

print(f'Normalized column names: {len(setB.difference(setA))}')

DF_raw.shape

Index(['language_id', 'name', 'last_update'], dtype='object')

Normalized column names: 0


(6, 3)

### 2. Eval misspelling in object non-date columns

In [5]:
for i in DF_raw.columns: 
    if (DF_raw[i].dtype == 'object') and ("date" not in i):
        print(i.upper(), ":")
        DF_raw[i]=DF_raw[i].apply(lambda a: a.strip()) ## Remove leading and trailing spaces 
        print(DF_raw[[i]].value_counts())
        print("\n")
    else:
        print(i.upper(), " ---> Non object-Column or explicit date reference")
        pass

LANGUAGE_ID  ---> Non object-Column or explicit date reference
NAME :
name    
English     1
French      1
German      1
Italian     1
Japanese    1
Mandarin    1
dtype: int64


LAST_UPDATE  ---> Non object-Column or explicit date reference


### 3. eval/drop duplicates without index_id

In [6]:
listtest=["film_id"]
DF_raw_wo1=mlg.DF_wo_colX(DF_raw, listtest)

print("\n")

DF1=DF_raw.copy()
if any(DF_raw_wo1.duplicated()): # there are duplicates!
    print("DUPLICATED?", any(DF_raw_wo1.duplicated()))
    # Find the index positions of duplicate rows in the subset of columns
    dup_ind = DF_raw_wo1.duplicated()

    # Show the duplicate rows
    display(DF1[dup_ind])

    # Display the index positions where rows are duplicated
    nondup_ind = dup_ind[~dup_ind].index
    DF1 = DF1.iloc[nondup_ind]

    # Reset the index to have continuous index values
    DF1.reset_index(drop=True, inplace=True) 
else:
    print("NO HIDDEN DUPLICATES")
    pass


display(DF_raw.shape)
display(DF1.shape)




NO HIDDEN DUPLICATES


(6, 3)

(6, 3)

### 4. check NAs again

- By column
- By row

In [16]:
display(mlg.na_absperc(DF1)) # by cols
display(mlg.na_absperc(DF1.T)) # by rows


,abs_NA,perc_NA


,abs_NA,perc_NA


### 5. unique/freq ratio

In [17]:
display(mlg.categ_summ(DF1).sort_values("resto_per", ascending =True))

,count,unique,top,freq,unicount_ratio,resto_abs,resto_per
last_update,6,1,2006-02-15 05:02:19,6,0.166667,0,0.0
name,6,6,English,1,1.0,5,83.333333


### 6. data types

In [18]:
DF1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   language_id  6 non-null      int64 
 1   name         6 non-null      object
 2   last_update  6 non-null      object
dtypes: int64(1), object(2)
memory usage: 272.0+ bytes


### 7. Save the cleaned dataframe(s)

In [20]:
language1=DF1.copy()
#language1.to_csv('../../data/clean/language1.csv', index=False)